# ETL for Travel Tips Dashboard

**Data Source:**  [Visual Crossing Corporation](https://www.visualcrossing.com/)


* Historical weather data was downloaded for five chosen cities plus Indianapolis for years 2010 - 2019 inclusive.   
* Due to limitation to the number of results per query, we downloaded data to 4 CSV files.
* The files were loaded to Pandas dataframe and then joined.
* We removed not needed columns and sorted dataframe per location and date in ascending order.
* The index was named so it could be used as a primary key in a database table.
* **Pandabase** was used to create SQLite database from our dataframe.
* We checked the database with SQLAlchemy by running several queries.


#### Import Dependencies

In [1]:
import pandas as pd
import pandabase
import numpy as np
from pandas import read_csv
import datetime as dt

#### Read csv files

Files **df1**, **df2**, **df3** are for all 5 below listed cities for the stated range of years:

Chicago  
Las Vegas  
Los Angeles  
New York City  
Washington

The file **df4** is for Indianapolis, for all 10 years.

In [2]:
df1 = read_csv("./Resources/history_data_10-13.csv", encoding="ISO-8859-1")
df2 = read_csv("./Resources/history_data_14-17.csv", encoding="ISO-8859-1")
df3 = read_csv("./Resources/history_data_18-19.csv", encoding="ISO-8859-1")
df4 = read_csv("./Resources/history_data_Indy_10-19.csv", encoding="ISO-8859-1")

In [3]:
print(df1.shape)
df1.head(2)

(7305, 16)


,Location,Address,Resolved Address,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow Depth,Wind Speed,Wind Gust,Cloud Cover,Relative Humidity,Conditions
0,Chicago,"Chicago, IL, US",NaN,2010-01-01,18.1,7.8,12.6,-7.3,NaN,0.0,2.0,16.0,NaN,262.4,64.96,"Snow, Overcast"
1,Chicago,"Chicago, IL, US",NaN,2010-01-02,12.4,4.8,8.5,-11.5,NaN,0.0,2.0,17.5,NaN,260.0,58.81,"Snow, Overcast"


In [4]:
print(df2.shape)
df2.head(2)

(7305, 16)


,Location,Address,Resolved Address,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow Depth,Wind Speed,Wind Gust,Cloud Cover,Relative Humidity,Conditions
0,Chicago,"Chicago, IL, US",NaN,2014-01-01,22.4,12.8,19.9,1.8,NaN,0.1,3.0,16.5,NaN,90.9,82.97,"Snow, Overcast"
1,Chicago,"Chicago, IL, US",NaN,2014-01-02,21.7,5.1,16.3,-10.4,NaN,0.1,7.2,19.0,31.1,70.3,76.11,"Snow, Partially cloudy"


In [5]:
print(df3.shape)
df3.head(2)

(3650, 16)


,Location,Address,Resolved Address,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow Depth,Wind Speed,Wind Gust,Cloud Cover,Relative Humidity,Conditions
0,Chicago,"Chicago, IL, US",NaN,2018-01-01,4.7,-5.2,-1.0,-28.0,NaN,0.0,2.0,25.4,19.7,0.6,63.31,"Snow, Clear"
1,Chicago,"Chicago, IL, US",NaN,2018-01-02,7.7,-6.0,0.9,-25.6,NaN,0.0,2.0,22.7,21.9,1.8,63.59,"Snow, Clear"


In [6]:
print(df4.shape)
df4.head(2)

(3652, 16)


,Location,Address,Resolved Address,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow Depth,Wind Speed,Wind Gust,Cloud Cover,Relative Humidity,Conditions
0,Indianapolis,"Indianapolis, IN, US",NaN,2010-01-01,18.6,8.9,13.5,-5.2,NaN,0.0,NaN,16.1,NaN,42.6,71.55,Partially cloudy
1,Indianapolis,"Indianapolis, IN, US",NaN,2010-01-02,16.6,2.0,9.8,-12.2,NaN,0.0,NaN,17.9,NaN,36.9,70.72,Partially cloudy


In [7]:
# Join dataframes
df = pd.concat([df1, df2, df3, df4]).reset_index(drop=True)
print(df.shape)
df.head(2)

(21912, 16)


,Location,Address,Resolved Address,Date time,Maximum Temperature,Minimum Temperature,Temperature,Wind Chill,Heat Index,Precipitation,Snow Depth,Wind Speed,Wind Gust,Cloud Cover,Relative Humidity,Conditions
0,Chicago,"Chicago, IL, US",NaN,2010-01-01,18.1,7.8,12.6,-7.3,NaN,0.0,2.0,16.0,NaN,262.4,64.96,"Snow, Overcast"
1,Chicago,"Chicago, IL, US",NaN,2010-01-02,12.4,4.8,8.5,-11.5,NaN,0.0,2.0,17.5,NaN,260.0,58.81,"Snow, Overcast"


#### Work with Columns

In [8]:
# Rename columns
df.columns = ['location', 'address', 'resolvedAddress', 'date', 'maxt', 'mint', 'temp', 'windchill', 
              'heatindex', 'precip', 'snowdepth', 'wspd', 'wgust', 'cloudcover', 'humidity', 'conditions']
df.head(2)

,location,address,resolvedAddress,date,maxt,mint,temp,windchill,heatindex,precip,snowdepth,wspd,wgust,cloudcover,humidity,conditions
0,Chicago,"Chicago, IL, US",NaN,2010-01-01,18.1,7.8,12.6,-7.3,NaN,0.0,2.0,16.0,NaN,262.4,64.96,"Snow, Overcast"
1,Chicago,"Chicago, IL, US",NaN,2010-01-02,12.4,4.8,8.5,-11.5,NaN,0.0,2.0,17.5,NaN,260.0,58.81,"Snow, Overcast"


In [9]:
# Choose features 
df = df.iloc[:,[0, 3, 4, 5, 6, 9, 11, 13, 14]]
df.head(2)

,location,date,maxt,mint,temp,precip,wspd,cloudcover,humidity
0,Chicago,2010-01-01,18.1,7.8,12.6,0.0,16.0,262.4,64.96
1,Chicago,2010-01-02,12.4,4.8,8.5,0.0,17.5,260.0,58.81


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21912 entries, 0 to 21911
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    21912 non-null  object 
 1   date        21912 non-null  object 
 2   maxt        21912 non-null  float64
 3   mint        21912 non-null  float64
 4   temp        21912 non-null  float64
 5   precip      21912 non-null  float64
 6   wspd        21912 non-null  float64
 7   cloudcover  21912 non-null  float64
 8   humidity    21912 non-null  float64
dtypes: float64(7), object(2)
memory usage: 1.5+ MB


In [11]:
# Sort dataframe and reset index
df = df.sort_values(['location', 'date'], ascending=[True, True]).reset_index(drop=True)
df

,location,date,maxt,mint,temp,precip,wspd,cloudcover,humidity
0,Chicago,2010-01-01,18.1,7.8,12.6,0.0,16.0,262.4,64.96
1,Chicago,2010-01-02,12.4,4.8,8.5,0.0,17.5,260.0,58.81
2,Chicago,2010-01-03,18.9,4.1,11.8,0.0,15.9,262.0,61.02
3,Chicago,2010-01-04,18.6,10.3,14.7,0.0,14.8,263.6,67.07
4,Chicago,2010-01-05,23.3,16.6,18.9,0.0,13.9,265.9,67.32
...,...,...,...,...,...,...,...,...,...
21907,Washington,2019-12-27,52.7,39.5,45.3,0.0,8.3,19.2,91.24
21908,Washington,2019-12-28,64.4,37.8,49.4,0.0,7.5,16.9,79.89
21909,Washington,2019-12-29,52.8,46.6,49.7,0.6,6.1,20.7,90.54
21910,Washington,2019-12-30,65.3,46.4,54.7,0.3,11.5,19.3,92.03


#### Create SQLite database from dataframe

In [12]:
# Name the index; needed for pandabase
# Index must be named to be used as PK
df.index.name = 'id'
df.index

RangeIndex(start=0, stop=21912, step=1, name='id')

In [13]:
df.head(2)

,location,date,maxt,mint,temp,precip,wspd,cloudcover,humidity
id,,,,,,,,,
0,Chicago,2010-01-01,18.1,7.8,12.6,0.0,16.0,262.4,64.96
1,Chicago,2010-01-02,12.4,4.8,8.5,0.0,17.5,260.0,58.81


In [14]:
# Set database variables
db_conn = 'sqlite:///./Resources/weather_history.sqlite'
sqlite_table = 'measurement'

In [15]:
# Run pandabase, create sqlite db and populate table
pandabase.to_sql(df, table_name=sqlite_table, con=db_conn)

Table('measurement', MetaData(bind=None), Column('id', Integer(), table=<measurement>, primary_key=True, nullable=False), Column('location', String(), table=<measurement>), Column('date', String(), table=<measurement>), Column('maxt', Float(), table=<measurement>), Column('mint', Float(), table=<measurement>), Column('temp', Float(), table=<measurement>), Column('precip', Float(), table=<measurement>), Column('wspd', Float(), table=<measurement>), Column('cloudcover', Float(), table=<measurement>), Column('humidity', Float(), table=<measurement>), schema=None)

#### Check the database with SQLAlchemy

In [16]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [17]:
engine = create_engine(db_conn)

In [18]:
# Reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [19]:
# We can view all of the classes that automap found
Base.classes.keys()

['measurement']

In [20]:
# Save references to each table
Measurement = Base.classes.measurement

#### Run some queries to test

In [21]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [22]:
list(session.query(Measurement).first().__dict__.keys())[1:]

['maxt',
 'id',
 'temp',
 'wspd',
 'humidity',
 'location',
 'mint',
 'date',
 'precip',
 'cloudcover']

In [23]:
locs = session.query(Measurement.location).distinct().all()
print(locs)

[('Chicago',), ('Indianapolis',), ('Las Vegas',), ('Los Angeles',), ('New York City',), ('Washington',)]


In [24]:
[r.location for r in session.query(Measurement.location).distinct()]

['Chicago',
 'Indianapolis',
 'Las Vegas',
 'Los Angeles',
 'New York City',
 'Washington']

In [25]:
res = session.query(Measurement.location, Measurement.date, Measurement.temp).\
            filter(Measurement.location == 'New York City').\
            filter(Measurement.date >= '2015-06-12').\
            filter(Measurement.date <= '2015-06-18').\
            order_by(Measurement.date).all()
print(res)

[('New York City', '2015-06-12', 81.3), ('New York City', '2015-06-13', 80.3), ('New York City', '2015-06-14', 77.0), ('New York City', '2015-06-15', 70.0), ('New York City', '2015-06-16', 71.0), ('New York City', '2015-06-17', 72.4), ('New York City', '2015-06-18', 66.8)]


In [26]:
date_list = np.ravel(session.query(Measurement.date).all())
date_list

array(['2010-01-01', '2010-01-02', '2010-01-03', ..., '2019-12-29',
       '2019-12-30', '2019-12-31'], dtype='<U10')

In [ ]:
session.close()